In [ ]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [ ]:
extracted_data = load_pdf_file(data="E:/ENd to end proects/ChatbotNLP/Data/")

In [ ]:
extracted_data[0]

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [1]:
%reset -f

In [ ]:
text_chunk=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunk))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    emebeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-miniLM-L6-V2')
    return emebeddings
embeddings_model=download_hugging_face_embeddings()

In [ ]:
import os
from dotenv import find_dotenv,load_dotenv
dotenv_path=find_dotenv()
load_dotenv(dotenv_path)
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [ ]:
import os
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")
HUGGINGFACE_API_KEY=os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
import os
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="nlpchatbot"
pc.create_index(name=index_name,dimension=384,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1"))

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(text_chunk,embeddings_model,index_name=index_name)

In [ ]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})
retrieved_docs=retriever.invoke("what is Semantic ANalysis")
retrieved_docs

In [ ]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [ ]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=50)

In [ ]:
#from langchain_huggingface import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
repo_id="mistralai/Mistral-7B-Instruct-v0.2"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=128,temperature=0.7,huggingfacehub_api_token=HUGGINGFACE_API_KEY)

In [ ]:
llm.invoke("what is nlp")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt=("You are an assistant for question answering tasks."
               "Use the following pieces of reetrieved context to answer"
               "the question. If you dont know the answer say that you"
               "don't know.Use three sentences maximun and keep the answer concise."
               "\n\n"
               "{context}"
               )

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
         
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"who is pamuri"})
print(response["answer"])